[![LibKet](../images/LibKet.png)](https://gitlab.com/libket/LibKet)
**LibKet - The Quantum Expression Template Library.**
- Repository:    https://gitlab.com/libket/LibKet/
- Documentation: https://libket.readthedocs.io/
- API docs:      https://libket.gitlab.io/LibKet/

***

# Tutorial \#3: Hands-on Scientific Computing with LibKet - Part 2

In this tutorial you will learn to
1. write quantum expressions with for-loops
2. build quantum algorithms from components

## Getting started
Let's include **LibKet**'s main headerfile, import its namespaces, and inject the code for displaying images. This can take some time, stay tuned.

In [ ]:
#include "LibKet.hpp"
using namespace LibKet;
using namespace LibKet::circuits;
using namespace LibKet::filters;
using namespace LibKet::gates;

A common building block in many quantum algorithms is the [Quantum Fourier Transform](https://en.wikipedia.org/wiki/Quantum_Fourier_transform) (QFT) and its inverse QFT$^\dagger$

![QFT](../images/qft_circuit.png)

where **CRk** denotes the **controlled phase shift** gate, ``crk<k>(ctrl, target)``, with angle $\theta = \pi/2^k$ (radians). This gate is particularly useful for implementing the QFT as the rotation angles vary with the distance between the control and the target qubit.

**LibKet** has ready-to-use quantum expressions ``qft(...)`` and ``qftdag(...)``. Let's start with the former one.

In [ ]:
auto expr = LibKet::circuits::qft<LibKet::circuits::QFTMode::noswap>(init());

QDevice<QDeviceType::qiskit_qasm_simulator, 4> qiskit;
std::cout << qiskit(expr) << std::endl;

Don't be surprised to see ``cu1(theta) q[i], q[j]`` instead of ``crk``. This is specific [OpenQASM](https://en.wikipedia.org/wiki/OpenQASM) code, one of the many quantum assembly languages, and it does not support the **CRk** gate natively. **LibKet** frees us from these technical subleties and allows us to write backend-agnostic code.

The pre-implemented ``qft<mode>(...)`` function makes use of another advanced **LibKet** feature.

## Compile-time for loops with ``static_for()``

The generic interface of the ``static_for()`` function reads
```cpp
template<index_t for_start,
         index_t for_end,
         index_t for_step,
         template<index_t start, index_t end, index_t step, index_t index>
         class functor,
         typename functor_return_type,
         typename... functor_types>
inline auto
static_for(functor_return_type&& functor_return_arg,
           functor_types&&... functor_args)
```
Let's start with a small
####  Example
We first create a **functor**, which represents the loop's body

In [ ]:
template<index_t start, index_t end, index_t step, index_t index>
struct ftor 
{
    template<typename  Expr>
    inline constexpr auto operator()(Expr&& expr) noexcept 
    {
        // Returns the controlled phase shift gate with angle
        // theta = pi/2^(index+1) between qubits index and index+1
        return crk<index+1>(sel<index>  (gototag<0>()),
                            sel<index+1>(gototag<0>(expr))
                           );
    }
};

To *loop* through this functor (at compile time) we call the ``utils::static_for<start, end, step, body>(...)`` function as follows. Note the usefulness of the ``tag``/``gototag`` mechanism to restore the original filter settings easily

In [ ]:
auto expr = utils::static_for<0,4,1,ftor>(tag<0>(init()));

QDevice<QDeviceType::qiskit_qasm_simulator, 4> qiskit;
std::cout << qiskit(expr) << std::endl;

#### Nested loops
Compile-time for loops can even be nested as it is done in our QFT implementation.

Here, the outer loop (``qft_loop_outer``) repeatedly calls the inner loop (``qft_loop_inner``) with different values of the ``start`` index

In [ ]:
template<index_t start, index_t end, index_t step, index_t index>
struct qft_loop_inner
{
    template<typename Expr0, typename Expr1>
    inline constexpr auto operator()(Expr0&& expr0, Expr1&& expr1) noexcept
    {
        return crk<index-start+2>(sel<index  >(gototag<0>(expr0)),
                                  sel<start-1>(gototag<0>(expr1))
                                 );
    }
};

template<index_t start, index_t end, index_t step, index_t index>
struct qft_loop_outer
{
    template<typename Expr0, typename Expr1>
    inline constexpr auto operator()(Expr0&& expr0, Expr1&& expr1) noexcept
    {
        return utils::static_for<index+1, end, 1, qft_loop_inner>
            (h(sel<index>(gototag<0>(expr0))),
             gototag<0>(expr1)
            );
    }
};

Armed with the nested compile-time for-loop we can implement the expression for the QFT as a one-liner

In [ ]:
auto expr = utils::static_for<0,3,1,qft_loop_outer>(tag<0>(init()), tag<0>());

QDevice<QDeviceType::qiskit_qasm_simulator, 4> qiskit;
std::cout << qiskit(expr) << std::endl;

What happens if you move the ``init()`` from the first to the second argument?

#### Execise: ``allswap()``
Now it's your turn. Write a functor and a compile-time for-loop that ``swap``s $\lvert q_0\rangle$ with $\lvert q_n\rangle$, $\lvert q_1\rangle$ with $\lvert q_{n-1}\rangle$, etcetera. 

Apply your implementation *after* the QFT and compare the generated OpenQASM code with our reference implementation
```cpp
auto expr = LibKet::circuits::qft(...);
```
or
```cpp
auto expr = LibKet::circuits::allswap(LibKet::circuits::qft<LibKet::circuits::QFTMode::noswap>(...));
```